In [1]:
# 107303538_1209_hw12_2

In [2]:
import numpy as np 
import torch
import torch.nn as nn

In [3]:
from google.colab import drive # 連接到google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


匯入labels.csv

In [4]:
path = './drive/MyDrive/Dog/labels.csv' 
dogs_numpy = np.loadtxt(path, dtype=str, delimiter=',', skiprows=1) # 處理 excel有內建分隔delimiter 去除第一行(id breed)
dogs_numpy, dogs_numpy.shape

(array([['000bec180eb18c7604dcecc8fe0dba07', 'boston_bull'],
        ['001513dfcb2ffafc82cccf4d8bbaba97', 'dingo'],
        ['001cdf01b096e06d78e9e5112d419397', 'pekinese'],
        ...,
        ['ffe2ca6c940cddfee68fa3cc6c63213f', 'airedale'],
        ['ffe5f6d8e2bff356e9482a80a6e29aac', 'miniature_pinscher'],
        ['fff43b07992508bc822f33d8ffd902ae', 'chesapeake_bay_retriever']],
       dtype='<U32'), (10222, 2))

匯入breed.txt

In [10]:
path = './drive/MyDrive/Dog/breed.txt' 
breed_numpy = np.loadtxt(path, dtype=str)
#breed_numpy

In [9]:
# 轉成list
#breed_list = list() # (可略) 空list為list()或[]
breed_list = list(breed_numpy)
#breed_list

In [13]:
# 轉成dic
dic = {} # (不可略) 空dict為dict()或{}
for i in range(len(breed_list)):
    dic[breed_list[i]] = i # dic[ key ] = value
#dic

處理dog_numpy(labels.csv)

In [16]:
dog_labels = list(dogs_numpy[:,-1]) # dogs_numpy為(10222, 2) 10222總狗 橫向看為(id, breed) [:,-1]為取所有狗的breed而已
dog_labels = [dic[breed_name] for breed_name in dog_labels] # dic[breed_name]最後會產出value 即找dog_labels內對應到dict的標號
dog_labels
dog_t_labels = torch.tensor(dog_labels) # 轉成tensor
dog_t_labels

# 舉例: dogs_numpy的第一種狗'boston_bull' 在breed.txt中為第19個 故dic['boston_bull']=19 而dog_labels[0]和dog_t_labels[0]為19和tensor(19)

tensor([19, 37, 85,  ...,  3, 75, 28])

匯入train圖片

In [23]:
import os
dog_file_name = [i +'.jpg' for i in dogs_numpy[:,0]] # dogs_numpy[:,0]為id
dog_file_path = [os.path.join('./drive/MyDrive/Dog/train', i_jpg) # 取train中每一個圖片 os.path.join為合併路徑
          for i_jpg in dog_file_name] 
#dog_file_path

正規化

對所有資料集

In [ ]:
from torchvision import transforms
# 資料集的圖片都不一樣大小 故需要先把圖片變成一樣大 後來轉成tensor才可以算mean和std
preprocess_resize_tensor = transforms.Compose([transforms.Resize([100, 100]), # 變正方形
                        transforms.ToTensor()]) # 轉tensor

means = torch.zeros(3)
bias = torch.zeros(3)

for i in range (0, 10222):
  imgs_real = Image.open(dog_file_path[i]) # 用第一張圖片作為試驗 為真正的圖片
  imgs_tensor = preprocess_resize_tensor(imgs_real) # 處理圖片
  means += imgs_tensor.view(3, -1).mean(dim=1) # 正規化標準輸入為(channel, 圖片其他參數) 再對dim=1做mean或std
  bias += imgs_tensor.view(3, -1).std(dim=1)

means/len(dog_file_path), bias/len(dog_file_path) # (tensor([0.4766, 0.4523, 0.3923]), tensor([0.2205, 0.2154, 0.2137]))

對一張圖片

In [24]:
from PIL import Image # 匯入外部圖片
img_real = Image.open(dog_file_path[1]) # 用第一張圖片作為試驗 為真正的圖片
img_tensor = preprocess_resize_tensor(img_real) # 處理圖片
img_tensor # torch.Size([3, 224, 224])

# 保留三個RGB channels 剩下的皆合併 一張圖片的shape變成torch.Size([3, 50176]) 最後對三個不同的channel求mean和std
img_tensor.view(3, -1).mean(dim=1), img_tensor.view(3, -1).std(dim=1) 

(tensor([0.4658, 0.3058, 0.1603]), tensor([0.2985, 0.2460, 0.1410]))

前處理preprocess(含正規化)

In [25]:
from torchvision import transforms, utils

preprocess = transforms.Compose([transforms.Resize([224, 224]), # 變正方形
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.4766, 0.4523, 0.3923], std=[0.2205, 0.2154, 0.2137]) # 正規化
])

定義loader

In [26]:
from PIL import Image # 匯入外部圖片
def default_loader(path):
    img_pil = Image.open(path) # 打開真正的圖片
    img_tensor = preprocess(img_pil) # 處理圖片
    return img_tensor

定義Dataset

In [31]:
# 定義Dataset類別 (init / getitem / len)
from torch.utils.data import Dataset, DataLoader

class TrainSet(Dataset):
  def __init__(self, load=default_loader): # 初始化
      self.images = dog_file_path # dog_file_path為照片的完整路徑
      self.target = dog_t_labels # 真正的狗的labels
      self.loader = default_loader
  
  def __getitem__(self, index): # 內建getitem的規則(有點像是打開一張圖片的方法)
      fn = self.images[index] # 也就是dog_file_path[1]
      img = self.loader(fn) # 輸出真正的圖片 並前處理後 返回其tensor
      target = self.target[index] # dog_t_labels[1] 真正的labels
      return img, target

  def __len__(self):
    return len(self.images) # 資料集長度

In [29]:
train_data = TrainSet() # 把model改名

定義DataLoader

In [35]:
trainloader = DataLoader(train_data, batch_size=16, shuffle=True) # DataLoader 為 BCHW
for imgs, labels in trainloader:
  batch_size = imgs.shape[0] # 一個batch
  break
imgs, labels

(tensor([[[[ 0.8798,  0.8442,  0.6841,  ...,  0.6130,  0.5063,  0.6130],
           [ 0.8442,  0.7553,  0.7375,  ...,  0.5241,  0.3284,  0.5063],
           [ 0.8798,  0.6841,  0.7553,  ...,  0.5063,  0.4529,  0.5774],
           ...,
           [ 0.4885,  0.7553,  0.4529,  ...,  0.7019,  0.6663,  0.7019],
           [ 0.3818,  0.3284,  0.5241,  ...,  0.7197,  0.7197,  0.7908],
           [ 0.4351,  0.3462,  0.5241,  ...,  0.8442,  0.9509,  0.9153]],
 
          [[ 1.0134,  0.9770,  0.8131,  ...,  0.7403,  0.6311,  0.7403],
           [ 0.9770,  0.8860,  0.8678,  ...,  0.6493,  0.4490,  0.6311],
           [ 1.0134,  0.8131,  0.8860,  ...,  0.6311,  0.5765,  0.7039],
           ...,
           [ 0.6129,  0.8860,  0.5765,  ...,  0.8313,  0.7949,  0.8313],
           [ 0.5036,  0.4490,  0.6493,  ...,  0.8496,  0.8496,  0.9224],
           [ 0.5583,  0.4672,  0.6493,  ...,  0.9770,  1.0862,  1.0498]],
 
          [[ 1.3022,  1.2655,  1.1004,  ...,  1.0270,  0.9169,  1.0270],
           [ 